In [7]:
import os
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [8]:
class BloodCellDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_files = []
        self.labels = []

        for label, category in enumerate(['no_agglutination', 'agglutination']):
            category_dir = os.path.join(image_dir, category)
            for img_name in os.listdir(category_dir):
                img_path = os.path.join(category_dir, img_name)
                try:
                    Image.open(img_path)  # Try opening image to ensure it's valid
                    self.image_files.append(img_path)
                    self.labels.append(label)
                except Exception as e:
                    print(f"Error loading image {img_path}: {e}")
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert("L")
        label = self.labels[idx]
            
        if self.transform:
            image = self.transform(image)
            
        return image, label


transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Corrected normalization
])


In [9]:
train_data = BloodCellDataset('training', transform=transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

test_data = BloodCellDataset('Testing', transform=transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [10]:
class BloodAgglutinationCNN(nn.Module):
    def __init__(self):
        super(BloodAgglutinationCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)  
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)  

        self.fc1 = nn.Linear(32 * 32 * 32, 128)  
        self.fc2 = nn.Linear(128, 1)  

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 32 * 32)  
        x = F.relu(self.fc1(x))       
        x = torch.sigmoid(self.fc2(x)) 
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BloodAgglutinationCNN().to(device)
criterion = nn.BCELoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [32]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train() 
        running_loss = 0.0
        for images, labels in train_loader:
            if images is None or labels is None:
                continue 
            images, labels = images.to(device), labels.to(device).float()  
            
            optimizer.zero_grad()  

            outputs = model(images)  
            loss = criterion(outputs.squeeze(), labels)  
            loss.backward()  
            optimizer.step() 

            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

train_model(model, train_loader, criterion, optimizer)

def test_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).float()
            outputs = model(images)
            predicted = (outputs.squeeze() > 0.5).float()  # Classify as 1 if sigmoid output > 0.5
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on test data: {accuracy:.2f}%')

test_model(model, test_loader)


Epoch [1/10], Loss: 0.0000
Epoch [2/10], Loss: 0.0000
Epoch [3/10], Loss: 0.0001
Epoch [4/10], Loss: 0.0000
Epoch [5/10], Loss: 0.0000
Epoch [6/10], Loss: 0.0000
Epoch [7/10], Loss: 0.0000
Epoch [8/10], Loss: 0.0000
Epoch [9/10], Loss: 0.0000
Epoch [10/10], Loss: 0.0000
Accuracy on test data: 75.00%


In [30]:
def infer_single_image(model, image_path):
    image = Image.open(image_path).convert("L")  # Convert to grayscale
    transform = transforms.Compose([
        transforms.Resize((128, 128)),  # Resize to 128x128
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
    ])
    image = transform(image).unsqueeze(0)  # Add batch dimension (1, 1, 128, 128)

    image = image.to(device)

    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        output = model(image)
        prediction = (output.squeeze() > 0.5).float()  # Sigmoid threshold at 0.5

    if prediction.item() == 1:
        print("True - Agglutination occurred")
    else:
        print("False - No agglutination detected")

# Example usage of inference
image_path = 'testing/no_agglutination/no_agglutination1.png'  
infer_single_image(model, image_path)


False - No agglutination detected


In [31]:
def classify_image(model, image_path):
    """Classify a single image as having agglutination (True) or not (False)."""
    image = Image.open(image_path).convert("L")  
    transform = transforms.Compose([
        transforms.Resize((128, 128)),  
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))  
    ])
    image = transform(image).unsqueeze(0)  

    image = image.to(device)

    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        output = model(image)
        prediction = (output.squeeze() > 0.5).float()  
    
    return bool(prediction.item())  


def determine_blood_type_and_rh(model, folder_path):
    
    imageA_path = os.path.join(folder_path, 'imageA.png')
    imageB_path = os.path.join(folder_path, 'imageB.png')
    imageAB_path = os.path.join(folder_path, 'imageAB.png')
    imageD_path = os.path.join(folder_path, 'imageD.png')


    agglutination_A = classify_image(model, imageA_path)
    agglutination_B = classify_image(model, imageB_path)
    agglutination_AB = classify_image(model, imageAB_path)
    agglutination_D = classify_image(model, imageD_path)

  
    if agglutination_A and agglutination_B and agglutination_AB:
        blood_type = "IV"  # Blood type AB (IV)
    elif agglutination_A and agglutination_AB and not agglutination_B:
        blood_type = "II"  # Blood type A (II)
    elif agglutination_B and agglutination_AB and not agglutination_A:
        blood_type = "III"  # Blood type B (III)
    else:
        blood_type = "I"  # Blood type O (I)

    if agglutination_D:
        rh_factor = "Rh+"
    else:
        rh_factor = "Rh-"

    print(f"Determined Blood Type: {blood_type} {rh_factor}")
    return blood_type, rh_factor



folder_path = 'dataset/Patient2' 
determine_blood_type_and_rh(model, folder_path)

Determined Blood Type: I Rh+


('I', 'Rh+')

In [33]:
torch.save(model.state_dict(), 'blood_agglutination_cnn.pth')
model.load_state_dict(torch.load('blood_agglutination_cnn.pth'))
model.eval()  

C:\Users\chiza\AppData\Local\Temp\ipykernel_17052\2735385211.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('blood_agglutination_cnn.pt

BloodAgglutinationCNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=32768, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)